# Section 1: Data Generation
Generate Dataset using env and customized data generator

In [1]:
from simple_stru_sampler import make_generator_params
from rl4co.envs import SDVRPEnv
from rl4co.models.zoo import AttentionModel
import torch
import random
import numpy as np

seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

params = make_generator_params(
    num_loc=18,
    outer_size=(8, 6),
    inner_size=(4, 2),
    vgap_range=(1.0, 2.0),
    hgap_range=(0.3, 0.5),
)

env = SDVRPEnv(generator_params=params)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
td_init = env.reset(batch_size=100).to(device)

Check Dataset

In [2]:
# Extract coords, capacity and demand from td_init
def get_tensor(td, *keys):
    for k in keys:
        if k in td.keys():
            return td[k]
    raise KeyError(f"None of keys {keys} found in tensordict")

coords = get_tensor(td_init, "locs", "coords")          # [B, N+1, 2]
capacity = get_tensor(td_init, "capacity", "veh_capacity", "vehicle_capacity")  # [B] or [B,1]
demand = get_tensor(td_init, "demand")  * capacity      # [B, N]

B, Np1, _ = coords.shape
N = Np1 - 1

for b in range(B):
    depot_xy = coords[b, 0]                    # [2]
    cities_xy = coords[b, 1:1+N]               # [N, 2]
    demands_b = demand[b]                      # [N]
    cap_b = capacity[b].item() if capacity[b].ndim == 0 else float(capacity[b].squeeze().item())

    print(f"\n=== Sample {b+1} ===")
    print(f"vehicle_capacity: {cap_b:.1f}")
    print(f"depot: (x={depot_xy[0]:.4f}, y={depot_xy[1]:.4f})")
    for i in range(N):
        x, y = cities_xy[i]
        d = demands_b[i]
        print(f"city{i+1}: (x={x:.4f}, y={y:.4f}), demand={float(d):.1f}")


=== Sample 1 ===
vehicle_capacity: 100.0
depot: (x=6.1461, y=5.2430)
city1: (x=2.0905, y=3.7354), demand=10.0
city2: (x=2.5679, y=3.7354), demand=5.0
city3: (x=3.0453, y=3.7354), demand=5.0
city4: (x=3.5226, y=3.7354), demand=5.0
city5: (x=4.0000, y=3.7354), demand=10.0
city6: (x=4.4774, y=3.7354), demand=5.0
city7: (x=4.9547, y=3.7354), demand=5.0
city8: (x=5.4321, y=3.7354), demand=5.0
city9: (x=5.9095, y=3.7354), demand=10.0
city10: (x=2.0905, y=2.2646), demand=10.0
city11: (x=2.5679, y=2.2646), demand=5.0
city12: (x=3.0453, y=2.2646), demand=5.0
city13: (x=3.5226, y=2.2646), demand=5.0
city14: (x=4.0000, y=2.2646), demand=10.0
city15: (x=4.4774, y=2.2646), demand=5.0
city16: (x=4.9547, y=2.2646), demand=5.0
city17: (x=5.4321, y=2.2646), demand=5.0
city18: (x=5.9095, y=2.2646), demand=10.0

=== Sample 2 ===
vehicle_capacity: 100.0
depot: (x=1.1602, y=3.7228)
city1: (x=2.3431, y=3.6428), demand=20.0
city2: (x=2.7573, y=3.6428), demand=10.0
city3: (x=3.1715, y=3.6428), demand=10.0
ci

# Section 2: Checkpoint

## Adjust ckpt name

In [13]:
ckpt = "checkpoints/epoch_epoch=099.ckpt"
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AttentionModel.load_from_checkpoint(
    ckpt,
    strict=False,
    # map_location="cpu",
    load_baseline=False,
    weights_only=False,
)

## Greedy Results and Plot

In [14]:
policy = model.policy.to(device)
policy.eval()

out = model(td_init.clone(), decode_type ="greedy")
print(out["reward"].mean().item())
print(f"{out['reward'].mean().item():.4f}")
print(out["reward"])


-21.157304763793945
-21.1573
tensor([-18.1204, -23.5639, -14.3639, -20.5373, -19.0432, -21.0195, -16.9661,
        -28.7354, -16.9286, -24.4804, -11.5203, -18.0614, -20.5914, -20.9257,
        -15.6408, -18.7952, -22.4856, -19.5364, -23.0957, -16.8964, -27.0894,
        -19.2084, -20.0522, -26.9501, -27.6845, -21.0399, -23.5432, -19.8337,
        -24.4942, -15.5230, -14.4572, -22.5763, -22.6116, -20.2452, -27.4498,
        -17.4482, -18.7256, -22.1193, -24.4609, -23.4539, -28.3803, -16.1372,
        -20.3046, -13.2966, -14.7107, -24.5070, -15.9264, -23.2273, -15.9076,
        -20.9572, -21.7029, -26.3353, -21.6086, -25.4603, -22.1473, -29.7276,
        -25.5301, -25.2978, -19.1041, -27.2340, -25.1571, -31.1020, -20.8764,
        -15.0875, -27.0625, -21.1305, -14.5700, -19.1864, -19.6686, -28.7232,
        -21.2639, -17.5094, -24.1185, -23.1882, -16.4739, -20.0304, -18.3421,
        -17.5697, -15.4225, -18.8439, -18.9084, -22.2914, -18.7228, -23.5951,
        -25.1656, -15.7919, -24.761

## Plot

In [ ]:
import matplotlib.pyplot as plt
import itertools

out = model(td_init.clone(), decode_type="greedy")

actions = out["actions"].detach().cpu().numpy()   # [B, T]
coords_np = coords.detach().cpu().numpy()         # [B, N+1, 2]
demand_np = demand.detach().cpu().numpy()         # [B, N]

B, Np1, _ = coords_np.shape
N = Np1 - 1

# 颜色循环器（每条 tour 一个颜色）
color_cycle = itertools.cycle(
    ["C0", "C1", "C2", "C3", "C4", "C5", "C6"]
)

for b in range(B):
    fig, ax = plt.subplots(figsize=(5, 5))

    depot_xy = coords_np[b, 0]
    cities_xy = coords_np[b, 1:1+N]
    demands_b = demand_np[b]

    # 画 depot & customers
    ax.scatter(depot_xy[0], depot_xy[1],
               marker="s", s=120, c="red")
    # ax.scatter(cities_xy[:, 0], cities_xy[:, 1],
    #            c="C0", s=40, label="Tour 1")

    # 标号
    for i, (x, y) in enumerate(cities_xy, start=1):
        ax.text(x, y, f"{i}", fontsize=8, ha="center", va="bottom")
    # -------- 核心：按 depot (=0) 切分 tour，并画回程 --------
    tour = actions[b]
    tour = tour[tour >= 0]   # 去 padding

    current_color = next(color_cycle)
    prev = 0   # 从 depot 开始

    for node in tour:
        if node == 0:
            # 回到 depot：先画上一站到 depot 的线
            if prev != 0:
                x_prev, y_prev = coords_np[b, prev]
                x_dep, y_dep   = coords_np[b, 0]
                ax.plot(
                    [x_prev, x_dep],
                    [y_prev, y_dep],
                    color=current_color,
                    linewidth=2,
                    alpha=0.9
                )
            # 重新从 depot 出发，换一条 tour 的颜色
            prev = 0
            current_color = next(color_cycle)
            continue

        # 正常从 prev -> node
        x_prev, y_prev = coords_np[b, prev]
        x_curr, y_curr = coords_np[b, node]

        ax.plot(
            [x_prev, x_curr],
            [y_prev, y_curr],
            color=current_color,
            linewidth=2,
            alpha=0.9
        )

        prev = node

    # 如果最后一段没有显式回到 depot，再补一条 prev -> depot
    if prev != 0:
        x_prev, y_prev = coords_np[b, prev]
        x_dep, y_dep   = coords_np[b, 0]
        ax.plot(
            [x_prev, x_dep],
            [y_prev, y_dep],
            color=current_color,
            linewidth=2,
            alpha=0.9
        )



        prev = node

    ax.set_title(f"Instance {b+1}, reward = {out['reward'][b].item():.4f}")
    ax.set_aspect("equal", adjustable="box")
    ax.grid(True, linestyle="--", alpha=0.3)
    ax.legend()

    plt.show()


# Section 3: Time for Greedy and Sampling Results

## Greedy

### time for Greedy

Greedy 1 instance per batch

In [15]:
import time

policy = model.policy.to(device)
policy.eval()

B = td_init.batch_size[0]

all_rewards = []
all_times = []
all_tours = []

for b in range(B):
    td_single = td_init[b:b+1].clone().to(device)
    start_t = time.perf_counter()

    out = model(td_single, decode_type="greedy")

    end_t = time.perf_counter()
    elapsed = end_t - start_t

    reward = out["reward"].item()
    tour = out["actions"][0].tolist()

    all_rewards.append(reward)
    all_times.append(elapsed)
    all_tours.append(tour)

    print(f"\n=== Sample {b+1} ===")
    print(f"Reward: {reward:.4f}")
    print(f"Time used: {elapsed:.4f} s")
    print(f"Tour (node visit order): {tour}")



=== Sample 1 ===
Reward: -18.1204
Time used: 0.1083 s
Tour (node visit order): [18, 17, 16, 15, 14, 13, 12, 11, 10, 1, 2, 3, 4, 5, 6, 0, 6, 7, 8, 9]

=== Sample 2 ===
Reward: -23.5639
Time used: 0.1159 s
Tour (node visit order): [5, 6, 7, 8, 9, 18, 17, 0, 16, 15, 14, 13, 12, 11, 10, 0, 1, 2, 3, 4]

=== Sample 3 ===
Reward: -14.3639
Time used: 0.0969 s
Tour (node visit order): [18, 17, 16, 15, 14, 13, 12, 11, 10, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

=== Sample 4 ===
Reward: -20.5373
Time used: 0.1456 s
Tour (node visit order): [6, 7, 8, 9, 18, 17, 16, 15, 14, 13, 0, 1, 2, 3, 4, 5, 13, 12, 11, 10]

=== Sample 5 ===
Reward: -19.0432
Time used: 0.1007 s
Tour (node visit order): [5, 6, 7, 8, 9, 18, 17, 16, 15, 14, 13, 12, 11, 10, 0, 1, 2, 3, 4]

=== Sample 6 ===
Reward: -21.0195
Time used: 0.0914 s
Tour (node visit order): [5, 6, 7, 8, 9, 18, 17, 16, 15, 14, 13, 12, 11, 10, 0, 1, 2, 3, 4]

=== Sample 7 ===
Reward: -16.9661
Time used: 0.1187 s
Tour (node visit order): [13, 12, 11, 10, 1, 2, 3, 4,

In [16]:
print("\n===== Summary =====")
print(f"Avg reward = {sum(all_rewards)/len(all_rewards):.4f}")
print(f"Total time = {sum(all_times):.4f} s")
print(f"Avg time per sample = {sum(all_times)/len(all_times):.4f} s")



===== Summary =====
Avg reward = -21.1573
Total time = 10.9951 s
Avg time per sample = 0.1100 s


Batch for 100 samples

In [ ]:
import time
torch.cuda.synchronize()

td_batch = td_init.clone().to(device)
B = td_batch.batch_size[0]

_ = model(td_batch[:4], decode_type="greedy")
torch.cuda.synchronize()

start = time.perf_counter()
out = model(td_batch, decode_type="greedy")
torch.cuda.synchronize()
end = time.perf_counter()

total_time = end - start
avg_time = total_time / B

print(f"RL policy: total time for {B} samples = {total_time:.4f}s")
print(f"RL policy: avg time per sample        = {avg_time:.6f}s")


## Sampling

### top_p

In [17]:
from rl4co.utils.ops import batchify

num_samples = 32
top_p = 1

bs = td_init.batch_size[0]
td_init_batched = batchify(td_init, num_samples)
out = model(td_init_batched.clone(), decode_type ="sampling",top_p=top_p)
rewards = torch.stack(out["reward"].split(bs), 1).max(1).values
print(rewards.mean().item())
print(f"{rewards.mean().item():.4f}")
print(rewards)


-21.136077880859375
-21.1361
tensor([-18.1204, -23.5639, -14.3639, -20.5373, -19.0432, -21.0195, -16.9661,
        -28.7354, -16.9286, -24.4804, -11.4992, -18.0614, -20.5914, -20.9257,
        -15.6408, -18.7952, -22.4856, -19.5364, -23.0957, -16.8964, -27.0894,
        -19.2084, -20.0522, -26.9501, -27.6845, -21.0399, -23.5432, -19.8337,
        -24.4942, -15.5230, -14.4572, -22.5763, -22.6116, -20.2452, -27.4498,
        -17.4482, -18.7256, -22.1193, -24.4609, -23.4539, -28.3803, -16.1372,
        -20.3046, -13.2966, -14.7107, -24.5070, -15.9264, -23.2273, -15.9076,
        -20.9572, -21.7029, -26.3353, -21.6086, -25.4603, -22.1473, -29.7276,
        -25.5301, -25.2081, -19.1041, -27.2340, -25.1571, -30.6578, -20.8764,
        -15.0875, -27.0625, -21.1305, -14.5700, -19.1864, -19.0645, -28.7232,
        -21.2639, -17.5094, -24.1185, -23.1882, -16.4739, -20.0304, -18.3421,
        -17.5697, -15.4225, -18.8439, -18.9084, -22.2914, -18.7228, -23.5951,
        -25.1656, -15.7919, -24.761

### top_p + temperature

In [18]:
import time
import torch
from rl4co.utils.ops import batchify

model.policy.temperature = 2.0  # 比如从 1.0 提到 2.0 或 3.0

num_samples = 32
top_p = 1

bs = td_init.batch_size[0]
td_init_batched = batchify(td_init, num_samples)

torch.cuda.synchronize()
start = time.perf_counter()

out = model(td_init_batched.clone(), decode_type="sampling", top_p=top_p)

torch.cuda.synchronize()
end = time.perf_counter()

rewards = torch.stack(out["reward"].split(bs), 1).max(1).values

print(f"Mean reward: {rewards.mean().item():.4f}")
print(f"Runtime: {(end - start) * 1000:.2f} ms")   # 毫秒
print(rewards)


Mean reward: -21.1251
Runtime: 647.78 ms
tensor([-18.1204, -23.4795, -14.3639, -20.5373, -19.0432, -21.0195, -16.9661,
        -28.7354, -16.9286, -24.4804, -11.1188, -17.9837, -20.5914, -20.9257,
        -15.6408, -18.7952, -22.4856, -19.5364, -23.0957, -16.8964, -27.0894,
        -19.2084, -20.0522, -26.9501, -27.6845, -21.0399, -23.5432, -19.8337,
        -24.4942, -15.5230, -14.4572, -22.5763, -22.6116, -20.2452, -27.4498,
        -17.4482, -18.7256, -22.1193, -24.4609, -23.4539, -28.3803, -16.1372,
        -20.3046, -13.2966, -14.7107, -24.5070, -15.9264, -23.2273, -15.9076,
        -20.9572, -21.7029, -26.3353, -21.6086, -25.4603, -22.1473, -29.7276,
        -25.5301, -25.2081, -19.1041, -27.2340, -25.1571, -30.6578, -20.8764,
        -15.0875, -27.0625, -21.1305, -14.5700, -19.1864, -18.8914, -28.7232,
        -21.2639, -17.5094, -24.1185, -23.1882, -16.4739, -20.0304, -18.3421,
        -17.5697, -15.4225, -18.8439, -18.9084, -22.2170, -18.7228, -23.5951,
        -25.1656, -15.7

### time for sampling

Sampling 1 for each

In [ ]:
import time
import torch
from rl4co.utils.ops import batchify

model.policy.temperature = 2.0
num_samples = 128
top_p = 1

bs = td_init.batch_size[0]   # 这里应该是 100
print("Total instances in td_init:", bs)

times = []
mean_rewards = []

for idx in range(bs):
    # 取出第 idx 个 instance，形状是 batch_size = 1
    td_one = td_init[idx:idx+1]

    # 对这个单个 instance 复制 num_samples 份
    td_one_batched = batchify(td_one, num_samples)   # => batch_size = 1 * 32

    torch.cuda.synchronize()
    start = time.perf_counter()

    out = model(td_one_batched.clone(), decode_type="sampling", top_p=top_p)

    torch.cuda.synchronize()
    end = time.perf_counter()

    run_time = end - start
    times.append(run_time)

    # 现在 reward 长度是 num_samples（因为只有 1 个原始 instance）
    # 从 32 个 sample 里取最好的一个
    rewards = out["reward"]   # shape: [num_samples] 或 [num_samples, 1]
    best_reward = rewards.max().item()
    mean_rewards.append(best_reward)

    print(
        f"Instance {idx+1:03d}: time = {run_time * 1000:.2f} ms, "
        f"best reward = {best_reward:.4f}"
    )

# 统计整体平均
mean_time = sum(times) / len(times)
mean_best_reward = sum(mean_rewards) / len(mean_rewards)

print("Best rewards for all instances:")
print(mean_rewards)   # 直接输出 Python list
print("=" * 60)
print(f"Average runtime over {bs} instances: {mean_time * 1000:.2f} ms")
print(f"Average best reward over {bs} instances: {mean_best_reward:.4f}")
print("=" * 60)
